In [17]:
import pandas as pd                     # Data manipulation
import numpy as np                      # Numeric computations
import statsmodels.api as sm            # Regression modeling
import statsmodels.formula.api as smf   # Regression modeling
import seaborn as sns                   # Visualizations
import matplotlib.pyplot as plt         # Visualizations
import plotly.express as px             # Visualizations

### Clean 'stores.xlsx'

In [18]:
# Import the stores.xlsx file
df_stores = pd.read_excel('stores.xlsx')

In [19]:
# Convert column headers to lower case
df_stores.columns = df_stores.columns.str.lower()
# Convert column data to lower case
df_stores = df_stores.map(lambda x: x.lower() if isinstance(x, str) else x)

### Clean 'products.xlsx'

In [20]:
# Import the products.xlsx file
df_products = pd.read_excel('products.xlsx')

In [21]:
# Convert column headers to lower case
df_products.columns = df_products.columns.str.lower()
# Convert column data to lower case
df_products = df_products.map(lambda x: x.lower() if isinstance(x, str) else x)

### Clean 'transactions.xlsx'

In [22]:
# Import the transactions.xlsx file
df_transactions = pd.read_excel('transactions.xlsx')

In [23]:
# Convert column headers to lower case
df_transactions.columns = df_transactions.columns.str.lower()
# Convert column data to lower case
df_transactions = df_transactions.map(lambda x: x.lower() if isinstance(x, str) else x)

### Remove oral hygiene products

In [24]:
# Create a clean copy of the dataframe to avoid chained assignment warnings
df_prod_cln = df_products[df_products['category'] != 'oral hygiene products'].copy()

In [25]:
# Remove 'oral hygiene products' based on UPCs in df_prod_cln (the products df only without the oral hygiene products)
keep_upcs = df_prod_cln['upc'].unique()

# Only keep the UPCs in df_prod_cln
df_trans_cln = df_transactions[df_transactions['upc'].isin(keep_upcs)]


### Standardize Product Size

In [26]:
# Function to extract numeric value and unit
def extract_size_details(size):
    if isinstance(size, str):  # Ensure it's a string before processing
        parts = size.strip().split(" ")  # Remove extra spaces
        if len(parts) >= 2:  # Ensure there's at least a value and unit
            try:
                value = float(parts[0])  # Convert first part to float
                unit = parts[1].lower()  # Standardize unit (lowercase)
                return pd.Series([value, unit])
            except ValueError:
                return pd.Series([None, None])  # Handle unexpected formats
    return pd.Series([None, None])



# Apply function to extract size and unit safely
df_prod_cln[['size_value', 'size_unit']] = df_prod_cln['product_size'].apply(extract_size_details)

# Function to convert all sizes to ML where applicable
def convert_to_ml(row):
    if pd.isna(row['size_value']) or pd.isna(row['size_unit']):
        return None  # Handle missing values
    unit = row['size_unit']
    value = row['size_value']
    
    if unit == 'ml':
        return value  # Already in ML
    elif unit == 'oz':
        return value * 29.5735  # Convert OZ to ML
    elif unit == 'lt':
        return value * 1000  # Convert LT to ML
    else:
        return None  # Keep CT and unknown units unchanged

# Apply function safely
df_prod_cln['size_in_ml'] = df_prod_cln.apply(convert_to_ml, axis=1)

# Debug: Check conversion results
print(df_prod_cln[['product_size', 'size_value', 'size_unit', 'size_in_ml']].head())

  product_size  size_value size_unit  size_in_ml
0        15 oz       15.00        oz  443.602500
1        15 oz       15.00        oz  443.602500
2        15 oz       15.00        oz  443.602500
6     12.25 oz       12.25        oz  362.275375
7        20 oz       20.00        oz  591.470000


In [27]:
# Count unique values in each column
unique_size_value = df_prod_cln['size_value'].nunique(dropna=True)
unique_size_unit = df_prod_cln['size_unit'].nunique(dropna=True)
unique_size_in_ml = df_prod_cln['size_in_ml'].nunique(dropna=True)

# Display results
print(f"Unique size_value count: {unique_size_value}")
print(f"Unique size_unit count: {unique_size_unit}")
print(f"Unique size_in_ml count: {unique_size_in_ml}")

Unique size_value count: 25
Unique size_unit count: 1
Unique size_in_ml count: 25


In [28]:
# Drop the original product_size column
df_prod_cln.drop(columns=['product_size'], inplace=True)

# Debug: Check if column is removed
print(df_prod_cln.head())  # Ensure product_size is gone

df_prod_cln.head()

          upc               description   manufacturer     category  \
0  1111009477    pl mini twist pretzels  private label   bag snacks   
1  1111009497         pl pretzel sticks  private label   bag snacks   
2  1111009507         pl twist pretzels  private label   bag snacks   
6  1111085319  pl honey nut toastd oats  private label  cold cereal   
7  1111085345            pl raisin bran  private label  cold cereal   

        sub_category  size_value size_unit  size_in_ml  
0           pretzels       15.00        oz  443.602500  
1           pretzels       15.00        oz  443.602500  
2           pretzels       15.00        oz  443.602500  
6  all family cereal       12.25        oz  362.275375  
7       adult cereal       20.00        oz  591.470000  


,upc,description,manufacturer,category,sub_category,size_value,size_unit,size_in_ml
0,1111009477,pl mini twist pretzels,private label,bag snacks,pretzels,15.00,oz,443.602500
1,1111009497,pl pretzel sticks,private label,bag snacks,pretzels,15.00,oz,443.602500
2,1111009507,pl twist pretzels,private label,bag snacks,pretzels,15.00,oz,443.602500
6,1111085319,pl honey nut toastd oats,private label,cold cereal,all family cereal,12.25,oz,362.275375
7,1111085345,pl raisin bran,private label,cold cereal,adult cereal,20.00,oz,591.470000


In [33]:
# Check missing values in all dataframes
print("Missing values in df_stores:")
print(df_stores.isnull().sum())
print('------------------------')
print("\nMissing values in df_prod_cln:")
print(df_prod_cln.isnull().sum())
print('------------------------')
print("\nMissing values in df_trans_cln:")
print(df_trans_cln.isnull().sum())

Missing values in df_stores:
store_id              0
store_name            0
city                  0
state                 0
msa                   0
segment               0
size                  0
avg_weekly_baskets    0
dtype: int64
------------------------

Missing values in df_prod_cln:
upc             0
description     0
manufacturer    0
category        0
sub_category    0
size_value      0
size_unit       0
size_in_ml      0
dtype: int64
------------------------

Missing values in df_trans_cln:
week_end_date      0
store_num          0
upc                0
units              0
visits             0
hhs                0
spend              0
price             10
base_price       173
feature            0
display            0
tpr_only           0
dtype: int64


In [32]:
# Drop 'parking' from df_stores (35% of df_stores have parking values.)
df_stores.drop(columns=['parking'], inplace=True)
df_stores.columns

Index(['store_id', 'store_name', 'city', 'state', 'msa', 'segment', 'size',
       'avg_weekly_baskets'],
      dtype='object')

In [36]:
# Define the Excel file name
# output_filename = "cleaned_data.xlsx"

# # Write the DataFrames to separate sheets in one Excel file
# with pd.ExcelWriter(output_filename, engine='auto') as writer:
#     df_stores.to_excel(writer, sheet_name="Stores", index=False)
#     df_prod_cln.to_excel(writer, sheet_name="Products", index=False)
#     df_trans_cln.to_excel(writer, sheet_name="Transactions", index=False)

# print(f"Excel file '{output_filename}' has been created successfully!")

Excel file 'cleaned_data.xlsx' has been created successfully!
